## 逐个翻译

In [ ]:
import requests
import os
import time

# === Step 1: 替换为你的 DeepL API Key ===
DEEPL_API_KEY = 'c6762b53-11b2-49c5-88a6-fba3c8fef7be:fx'  # <<< 替换这里

# === Step 2: 文件路径设置 ===
input_file = '/Users/elsa/Desktop/爬虫学习/0721学习/1900_2200.txt'
script_dir = os.path.dirname(os.path.abspath(__file__))
output_file = os.path.join(script_dir, 'translated_output_300.txt')

# === Step 3: 翻译函数（使用 DeepL） ===
def deepl_translate(text, target_lang='ZH'):
    url = 'https://api-free.deepl.com/v2/translate'
    params = {
        'auth_key': DEEPL_API_KEY,
        'text': text,
        'target_lang': target_lang
    }
    try:
        response = requests.post(url, data=params)
        result = response.json()
        if 'translations' in result:
            return result['translations'][0]['text']
        else:
            print("翻译失败:", result)
            return text
    except Exception as e:
        print("请求出错:", e)
        return text

# === Step 4: 逐行读取 + 翻译 + 写出 ===
with open(input_file, 'r', encoding='utf-8') as fin:
    lines = fin.readlines()

total_lines = len(lines)
with open(output_file, 'w', encoding='utf-8') as fout:
    for idx, line in enumerate(lines):
        parts = line.strip().split(',', 2)
        if len(parts) == 3:
            row, col, text = parts
            if text.strip():
                translated = deepl_translate(text)
                time.sleep(0.5)
            else:
                translated = ''
            fout.write(f"{row},{col},{translated}\n")
        else:
            fout.write(line)
        fout.flush()  # 立即刷新到文件
        os.fsync(fout.fileno())  # 强制写入磁盘

        # 显示进度条
        progress = (idx + 1) / total_lines * 100
        print(f"\r翻译进度：{progress:.2f}%", end='')

print(f"\n✅ 翻译完成，文件已保存到：{output_file}")


## 筛选没成功的翻译 重复操作

## 把翻译好的文件合并到源数据中

In [ ]:
import pandas as pd

# 路径设置
excel_path = '/Users/elsa/Desktop/爬虫学习/原始代码/苏州河评论.xlsx'  # 你的Excel文件
translated_txt = '/Users/elsa/Desktop/爬虫学习/0721_翻译/【全中文】translated_output_苏河.txt'
output_excel = '/Users/elsa/Desktop/爬虫学习/0721_翻译/最终版本/苏州河评论数据_带翻译.xlsx'

# 读Excel文件，假设有表头
df = pd.read_excel(excel_path)

# 读取翻译文件，构造字典 {(行号, 列号): 翻译内容}
translations = {}
with open(translated_txt, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(',', 2)
        if len(parts) == 3:
            row_str, col_letter, text = parts
            try:
                row = int(row_str)
                # Excel行号减去1对应pandas的行索引（假设Excel第一行为表头，数据从第2行起）
                pandas_row = row - 2  
                if pandas_row < 0 or pandas_row >= len(df):
                    # 跳过无效行
                    continue
                # F列对应标题列，G列对应内容列
                if col_letter == 'F':
                    translations[(pandas_row, '标题（F列）')] = text
                elif col_letter == 'G':
                    translations[(pandas_row, '内容（G列）')] = text
            except:
                pass

# 替换对应单元格
for (row_idx, col_name), value in translations.items():
    df.at[row_idx, col_name] = value

# 保存新的Excel文件
df.to_excel(output_excel, index=False)

print(f"✅ 写回完成，文件保存到：{output_excel}")
